# Experimenting with brute force engineering with most important variables

## Most important variables were:

#### Logistic Regression: V14, V12, V4, V17, V10, V8, V11, V16, V3, V1, V7, V18, V25
#### Random Forest: 'V17', 'V10', 'V12', 'V14', 'V16', 'V11', 'V9', 'V4', 'V18', 'V7','V26', 'V20'
#### Random Forest SMOTE: 'V10', 'V14', 'V12', 'V11', 'V17', 'V4', 'V16', 'V7', 'V18', 'V9', V20', 'V26'

In [17]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv ('../../data/creditcard_v2.csv')

In [3]:
most_important_variables = ['V10', 'V14', 'V12', 'V17', 'V16', 'V11', 'V7', 'V18', 'V4', 'V9']

In [4]:
df_most_important = df [most_important_variables]

In [5]:
df_features = df_most_important.copy (deep = True)

In [6]:
def ratios (df):
    for i in most_important_variables:
        for j in most_important_variables:
            if i==j:
                continue
            df [i + '/' + j] = df [i].div (df [j])
    return df

In [7]:
df_features = ratios (df_features)

In [8]:
df_features.shape

(283296, 100)

In [10]:
df_features.sample (5)

,V10,V14,V12,V17,V16,V11,V7,V18,V4,V9,...,V4/V9,V9/V10,V9/V14,V9/V12,V9/V17,V9/V16,V9/V11,V9/V7,V9/V18,V9/V4
229257,1.566889,-0.963503,-0.238949,-0.503161,-0.401793,-2.304551,0.579912,-0.554500,-0.147333,1.856433,...,-0.079363,1.184789,-1.926754,-7.769163,-3.689538,-4.620370,-0.805551,3.201231,-3.347938,-12.600260
169126,1.785092,0.023116,-1.423778,1.179105,-1.265111,0.229429,-1.037076,1.211691,-0.952455,-2.247304,...,0.423821,-1.258929,-97.218021,1.578409,-1.905940,1.776369,-9.795223,2.166961,-1.854684,2.359486
258977,1.072765,0.460964,-0.275629,0.160675,0.960424,1.409102,-0.446445,-0.868039,-0.831591,-0.680666,...,1.221731,-0.634497,-1.476615,2.469500,-4.236303,-0.708714,-0.483050,1.524636,0.784143,0.818511
22890,-0.100070,0.253189,-0.138911,0.192352,-0.567901,-1.034504,0.008190,-0.731103,1.329371,0.470640,...,2.824603,-4.703110,1.858848,-3.388057,2.446764,-0.828735,-0.454942,57.464226,-0.643739,0.354032
186993,-0.553186,-1.292063,0.429937,0.290073,0.269856,-0.515136,1.016864,-0.349438,-0.604071,-0.007615,...,79.330271,0.013765,0.005893,-0.017711,-0.026251,-0.028217,0.014782,-0.007488,0.021791,0.012606


(100,)

In [11]:
rf = RandomForestClassifier ()

In [55]:
X_train, X_test, y_train, y_test = train_test_split (df, df.Class, 
                                                    stratify = df.Class,
                                                    test_size = 0.25,
                                                    random_state = 0)

rf = RandomForestClassifier ()
model = rf.fit (X_train, y_train)

In [56]:
X_train_i, X_test_i, y_train_i, y_test_i = train_test_split (df_most_important, df.Class, 
                                                    stratify = df.Class,
                                                    test_size = 0.25,
                                                    random_state = 0)

rf = RandomForestClassifier ()
model_important = rf.fit (X_train_i, y_train_i)

In [57]:
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split (df_features, df.Class, 
                                                    stratify = df.Class,
                                                    test_size = 0.25,
                                                    random_state = 0)

rf = RandomForestClassifier ()
model_all_features = rf.fit (X_train_b, y_train_b)

In [13]:
from sklearn.metrics import precision_score, recall_score

## Normal dataset

In [58]:
precision_score (y_test, model.predict (X_test))

1.0

In [59]:
recall_score (y_test, model.predict (X_test))

1.0

## Brute force

In [60]:
precision_score (y_test_b, model_all_features.predict (X_test_b))

0.9090909090909091

In [61]:
recall_score (y_test_b, model_all_features.predict (X_test_b))

0.7317073170731707

## Most important

In [62]:
precision_score (y_test_i, model_important.predict (X_test_i))

0.9223300970873787

In [63]:
recall_score (y_test_i, model_important.predict (X_test_i))

0.7723577235772358

### Exploring variable importance

In [28]:
variable_coefficient = (
    pd.DataFrame (
        [df_most_important.columns, abs (model_important.feature_importances_)])
    .transpose ()
    .sort_values (by = 1, ascending = False))

In [29]:
variable_coefficient.head (20).reset_index(drop = True)

,0,1
0,V17,0.222552
1,V12,0.198274
2,V14,0.148295
3,V10,0.130022
4,V16,0.066594
5,V11,0.064038
6,V4,0.0466989
7,V7,0.0442519
8,V9,0.0423089
9,V18,0.0369653


In [ ]:
df_features.hist (figsize = (16,20), bins = 50, xlabelsize=8, ylabelsize=8);